In [1]:
from IPython.display import display, HTML, Image
display(HTML("<style>.container { width:85% !important; }</style>"))
%config IPCompleter.use_jedi=False

In [2]:
import torch
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
import pandas as pd
import re
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed, StoppingCriteria, StoppingCriteriaList
from transformers import pipeline

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
tokenizer = AutoTokenizer.from_pretrained("codeparrot/codeparrot-small")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("codeparrot/codeparrot-small")
# human_eval = load_dataset("openai_humaneval", split='test').filter(lambda x: len(x['canonical_solution'].split('\n'))<3)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
model = model.to(device)

In [10]:
EOF_STRINGS = ["\nclass", "\ndef", "\n@", "\nprint", "\nif", "\n>"]
EOF_SPLIT_REGEX = "(" + "|".join(EOF_STRINGS) + ")"

In [5]:
def generate_run(tokenizer,
                 model,
                 prompt,
                 temperature=0.6,
                 top_k=50,
                 top_p=1.0,
                 typical_p=1.0,
                 do_sample=False,
                 num_beams=1,
                 num_beam_groups=1,
                 num_return_sequences=1,
                 num_tokens_to_generate=128):
    
    len_prompt = len(tokenizer.tokenize(prompt))
    tokenized_promnpt = tokenizer(prompt, return_tensors='pt', padding=True)
    input_ids = tokenized_promnpt.input_ids.to(device)
    attention_mask = tokenized_promnpt.attention_mask.to(device)
    with torch.no_grad():
        generation = model.generate(inputs=input_ids,
                                    temperature=temperature,
                                    top_k=top_k,
                                    top_p=top_p,
                                    typical_p=typical_p,
                                    do_sample=do_sample, 
                                    num_beams=num_beams,
                                    num_beam_groups=num_beam_groups,
                                    max_length=len_prompt+num_tokens_to_generate, 
                                    num_return_sequences=num_return_sequences,
                                   pad_token_id=tokenizer.eos_token_id)
    generation = tokenizer.batch_decode(generation, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return {'cleaned_generation':[prompt+re.split(EOF_SPLIT_REGEX, i[len(prompt):])[0] for i in generation],
            'full_generation': generation,
            'temperature':temperature,
            'top_k': top_k,
            'top_p':top_p,
            'typical_p':typical_p,
            'do_sample':do_sample,
            'num_beams':num_beams,
            'num_beam_groups':num_beam_groups,
            'num_return_sequences':num_return_sequences,
            'num_tokens_to_generate':num_tokens_to_generate,
           }

In [20]:
temps = [round(x * 0.1, 1) for x in range(1, 51)]
top_p = [round(x * 0.1, 1) for x in range(1, 10)]
typical_p = [round(x * 0.1, 1) for x in range(1, 10)]

In [307]:
%%time
series_list = []
for p in typical_p:
    for row in tqdm(human_eval):
        generation = generate_run(tokenizer, model, row['prompt'], temperature=1.4, 
                                  top_k=50, top_p=1.0, typical_p=p, do_sample=True, 
                                  num_beams=1, num_beam_groups=1, num_return_sequences=200,
                                  num_tokens_to_generate=128)
        series_list.append(pd.Series({**row, **generation}))

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

CPU times: user 18min, sys: 7min 29s, total: 25min 29s
Wall time: 25min 28s


In [308]:
results = pd.DataFrame(series_list)
results.to_json('../runs/codeparrot-small/typical_p_sweep_temp_1.4.jsonl', lines=True, orient='records')

In [28]:
g['full_generation']

['\ndef sub(a,b):\n    return a-b\n\ndef mul(a,b):\n    return a*b\n\ndef div(a',
 '\ndef sub(a,b):\n    return a-b\n\ndef mul(a,b):\n    return a*b\n\ndef div(a']

In [26]:
g = generate_run(tokenizer=tokenizer,
                          model=model,
                          prompt="\ndef sub(a,b):",
                          temperature=t,
                          num_beams=10,
                          num_beam_groups=10,
                          num_return_sequences=2,
                          num_tokens_to_generate=25)

In [38]:
%%time
for typ_p in typical_p:
    g = generate_run(tokenizer=tokenizer,
                          model=model,
                          prompt="\ndef build_classifier(k, n):",
                          typical_p=typ_p,
                          do_sample=True,
                          num_return_sequences=3,
                          num_tokens_to_generate=50)
    for i in g['cleaned_generation']:
        print(f"{typ_p=}")
        print(i)
        print("*********")

typ_p=0.1

def build_classifier(k, n):
    return KNNClassifier(k=k, n=n, n_class=1, alpha=0.5, gamma=0.1)


*********
typ_p=0.1

def build_classifier(k, n):
    return KNNClassifier(k=k, n=n, n_classes=3, init='uniform',
                         subsample=0.5, alpha=0.1, eta=0.001,
                         alpha_init=0
*********
typ_p=0.1

def build_classifier(k, n):
    return np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
*********
typ_p=0.2

def build_classifier(k, n):
    return classifier.Classifier(n_estimators=n, max_depth=k,
                                min_samples_split=n, random_state=42)


*********
typ_p=0.2

def build_classifier(k, n):
    # Classifier
    n = n + 1
    n = n + 1
    # Classifier
    clf = LinearSVC(C=1.0, kernel='rbf', probability=True, tol=1e-4,
                    random_
*********
typ_p=0.2

def build_classifier(k, n):
    # type: (int, int) -> Dict[str, int]
    # This is the first time we run the classifier, and then 

In [22]:
for g in a['full_generation']:
    print(g)
    print('*********')


def sub(a,b):
    return a-b

def mul(a,b):
    return a*b

def div(a
*********

def sub(a,b):
    return a-b

def mul(a,b):
    return a*b

def div(a
*********
